<a href="https://colab.research.google.com/github/Thanat-Music/CNN-GUR/blob/main/CNN_GRU_signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !mv /content/kaggle.json ~/.kaggle/
# !kaggle competitions download -c uwb-pose-prediction
# !unzip /content/uwb-pose-prediction.zip
# # /content/drive/MyDrive/superAI3/signal_Hack 

In [ ]:
!pip install torch transformers datasets evaluate pillow==9.2.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler, Subset
from torchvision.datasets import ImageFolder
import torch as t
import torch.nn as nn
import math
from torch.autograd import Variable
import numpy as np


# Load Data

In [ ]:
!unzip /content/drive/MyDrive/SuperAi2/datasets/uwb-pose-prediction-v2-image-v1.zip

In [ ]:
train_files = glob.glob("/content/drive/MyDrive/SuperAi2/signal/signal_train/**/*.png")
test_files = glob.glob("/content/drive/MyDrive/SuperAi2/signal/signal_test/*.png")

NameError: ignored

In [ ]:
example = Image.open(train_files[0]).convert("RGB")
display(example)

# Preprocess

In [ ]:
transforms = {
    "train": T.Compose([
        T.Resize((224, 224), interpolation=T.InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=torch.tensor([0.4850, 0.4560, 0.4060]), std=torch.tensor([0.2290, 0.2240, 0.2250]))
    ]),
    "test": T.Compose([
        T.Resize((224, 224), interpolation=T.InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=torch.tensor([0.4850, 0.4560, 0.4060]), std=torch.tensor([0.2290, 0.2240, 0.2250]))
    ])
}

In [ ]:
# Define the dataset and transform
dataset = ImageFolder("/content/train", transform=transforms["train"])

# Define the train-test split ratio and the number of samples in the dataset
train_ratio = 0.8
num_samples = len(dataset)

# Define the number of samples in the train and test dataset
num_train_samples = int(0.8 * num_samples)
num_test_samples = num_samples - num_train_samples

train_dataset, eval_dataset = random_split(dataset, [num_train_samples, num_test_samples])

# Create the dataloaders for train and test using the train and test subset
# 73 is window number
train_loader = DataLoader(train_dataset, batch_size=73, shuffle=True)
val_loader = DataLoader(eval_dataset, batch_size=73, shuffle=True)

# Model

In [ ]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")

## GRU

In [ ]:
class GRUCell(nn.Module):

    """
    An implementation of GRUCell.

    """

    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):   
        x = x.view(-1, x.size(1))
        gate_x = self.x2h(x) 
        gate_h = self.h2h(hidden)

        gate_x = gate_x.squeeze()
        gate_h = gate_h.squeeze()

        i_r, i_i, i_n = gate_x.chunk(3, 1)
        h_r, h_i, h_n = gate_h.chunk(3, 1)

        resetgate = t.sigmoid(i_r + h_r)
        inputgate = t.sigmoid(i_i + h_i)
        newgate = t.tanh(i_n + (resetgate * h_n))
        
        hy = newgate + inputgate * (hidden - newgate)
        
        return hy

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(GRUModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
         
        # Number of hidden layers
        self.layer_dim = layer_dim
 
        self.gru_cell = GRUCell(input_dim, hidden_dim, layer_dim)
        
        
        self.fc = nn.Linear(hidden_dim, output_dim)

    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        #print(x.shape,"x.shape")100, 28, 28
        if t.cuda.is_available():
            h0 = Variable(t.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(t.zeros(self.layer_dim, x.size(0), self.hidden_dim))
         
       
        outs = []
        
        hn = h0[0,:,:]
        
        for seq in range(x.size(1)):
            hn = self.gru_cell(x[:,seq,:], hn) 
            outs.append(hn)
            

        out = outs[-1].squeeze()
        
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out

## CNN-GRU


In [ ]:
class CNN_GRU(nn.Module):
    def __init__(self, input_dim, hidden_size, output_dim, kernel_size=3, padding=1, num_layers=3):
        super(CNN_GRU, self).__init__()

        self.cnn_layer = nn.Sequential(
            nn.Conv2d(in_channels=input_dim, out_channels=32, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru_layer = nn.GRU(input_size =784, hidden_size =hidden_size, num_layers =num_layers,  batch_first=True)#,
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # CNN layer
        x = self.cnn_layer(x)

         # Reshape output for GRU layer
        # x = x.permute(0, 3, 1, 2)  # swap channel and sequence length dimensions
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, channels, height * width)

        # GRU layer
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Forward propagate GRU
        out, _ = self.gru_layer(x,h0)
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])

        return out

## F1 Macro score

In [ ]:
from sklearn.metrics import f1_score

def f1_macro(logits, targets):
    preds = t.argmax(logits, dim=1)
    return f1_score(targets, preds.cpu().numpy(), average='macro')

## Training

In [ ]:
hidden_dim = 128
num_layers = 1 #for GRU
input_dim = 3  # for RGB
output_dim = 7
lr = 0.0005

In [ ]:
model = CNN_GRU(input_dim, hidden_dim, output_dim ,num_layers=num_layers)
optim = t.optim.Adam(model.parameters(), lr=lr)
# loss_func = nn.BCELoss()
loss_func = t.nn.CrossEntropyLoss()

In [ ]:
epoch = 0
model = model.to(device)
while epoch <= 10:
    model.train()
    optim.zero_grad()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        pred = model(X_batch)
        loss_train = loss_func(pred, y_batch)
        loss_train.backward()
        optim.step()

    model.eval()
    with t.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            pred = model(X_batch)
            loss_val = loss_func(pred, y_batch)
            idx_pred = np.argmax(pred.cpu().detach().numpy(),axis=1)
            acc = f1_macro(pred.cpu(), y_batch.cpu())
    epoch += 1
    loss_train = loss_train.cpu().item()
    loss_val = loss_val.cpu().item()
    # if epoch % 10 == 0:
    print(f'Round: {epoch} || train_loss: {loss_train:.3f} || val_loss: {loss_val:.3f} || val_acc: {acc:.3f}')

Round: 1 || train_loss: 1.843 || val_loss: 1.809 || val_acc: 0.143
Round: 2 || train_loss: 1.872 || val_loss: 1.880 || val_acc: 0.177
Round: 3 || train_loss: 2.247 || val_loss: 1.853 || val_acc: 0.214
Round: 4 || train_loss: 1.459 || val_loss: 1.843 || val_acc: 0.270
Round: 5 || train_loss: 1.644 || val_loss: 1.634 || val_acc: 0.170
Round: 6 || train_loss: 1.569 || val_loss: 1.843 || val_acc: 0.316
Round: 7 || train_loss: 1.446 || val_loss: 1.690 || val_acc: 0.240


# Predict